Diego Jose Cantu Martinez A00816015
Actividad 3 | Aprendizaje supervisado y no supervisado

Introducción teórica


El aprendizaje supervisado es un tipo de aprendizaje  automatico donde se usan conjuntos de datos etiquetados para entrenar el modelo. Esto significa que cada input tiene un output deseado y el objetivo del modelo es devolver la misma salida y aprenderla.

Algoritmos representativos:
Regresión lineal: Predice un valor continuo basado en una o más variables independientes.

Regresión logística: Utilizado para clasificación binaria o multiclase.

Árboles de decisión: Algoritmo que divide los datos en subconjuntos basados en condiciones.

Random Forest: Conjunto de árboles de decisión que mejora precisión y reduce el sobreajuste.

Máquinas de vectores de soporte (SVM): Clasificación mediante hiperplanos óptimos.

Redes neuronales: Modelos inspirados en el cerebro humano para aprender relaciones complejas.

Algoritmos disponibles en PySpark (MLlib):
LinearRegression

LogisticRegression

DecisionTreeClassifier y DecisionTreeRegressor

RandomForestClassifier y RandomForestRegressor

GBTClassifier (Gradient Boosted Trees)

MultilayerPerceptronClassifier (una forma de red neuronal)

NaiveBayes

El aprendizaje no supervisado se refiere a el tipo de aprendizaje donde los valores de entrada no cuentan con valores de salida predeterminados. El objetivo del modelo es identificar patrones o agrupaciones de manera autonoma.

Algoritmos representativos:
Clustering (agrupamiento):

K-Means: Algoritmo popular que agrupa datos en K clústeres según su similitud.

DBSCAN: Agrupa puntos que están densamente conectados.

Jerárquico: Crea una jerarquía de clústeres.

Reducción de dimensionalidad:

PCA (Análisis de Componentes Principales): Reduce la dimensionalidad conservando la mayor varianza posible.

t-SNE: Proyección no lineal para visualización.

Algoritmos disponibles en PySpark (MLlib):
KMeans

BisectingKMeans (variante jerárquica)

GaussianMixture (modelo de mezcla gaussiana)

PCA (para reducción de dimensionalidad)

LDA (Latent Dirichlet Allocation, para análisis de temas en texto)



Selección de los datos

In [15]:
import requests
from pyspark.sql import SparkSession
# Descargar archivo Titanic CSV a disco local
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
r = requests.get(url)

# Guardar archivo localmente
with open("titanic.csv", "wb") as f:
    f.write(r.content)
df = spark.read.csv("titanic.csv", header=True, inferSchema=True)
df.show(5)




+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

Preparación de los datos

In [16]:
from pyspark.sql.functions import mean
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']
df = df.select(columns)

# Imputar valores nulos en Age y Fare con la media
age_mean = df.select(mean("Age")).first()[0]
fare_mean = df.select(mean("Fare")).first()[0]

df = df.fillna({"Age": age_mean, "Fare": fare_mean})

# Indexar la columna 'Sex'
indexer = StringIndexer(inputCol="Sex", outputCol="SexIndex")
df = indexer.fit(df).transform(df)

# Codificación One-Hot
encoder = OneHotEncoder(inputCols=["SexIndex"], outputCols=["SexVec"])
df = encoder.fit(df).transform(df)

feature_cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'SexVec']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df)





Preparación del conjunto de entrenamiento y prueba

In [17]:
# Dividir por clase para muestreo estratificado
df_class_0 = df.filter(df["Survived"] == 0)
df_class_1 = df.filter(df["Survived"] == 1)

# Fracción de prueba
test_frac = 0.3

# Separar entrenamiento y prueba para cada clase
train_0, test_0 = df_class_0.randomSplit([1 - test_frac, test_frac], seed=42)
train_1, test_1 = df_class_1.randomSplit([1 - test_frac, test_frac], seed=42)

# Unir ambas clases
train_df = train_0.union(train_1)
test_df = test_0.union(test_1)


# Conteo y proporción en train y test
print("Distribución en el conjunto de entrenamiento:")
train_df.groupBy("Survived").count().show()

print("Distribución en el conjunto de prueba:")
test_df.groupBy("Survived").count().show()

Distribución en el conjunto de entrenamiento:
+--------+-----+
|Survived|count|
+--------+-----+
|       0|  411|
|       1|  254|
+--------+-----+

Distribución en el conjunto de prueba:
+--------+-----+
|Survived|count|
+--------+-----+
|       0|  138|
|       1|   88|
+--------+-----+



Se decidió usar el 70% de los datos para entrenar el modelo y el 30% para probarlo. Esta forma de dividir los datos suele ser un estandar para los modelos supervisados. Con estos porcentajes le da al modelo suficientes datos para que pueda aprender bien y guarda una buena parte para probar si el modelo realmente funciona y no solo se aprendió de memoria los datos del entrenamiento.



Construcción de modelos de aprendizaje supervisado y no supervisado

Aprendizaje Supervisado

In [21]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Definir modelo
lr = LogisticRegression(featuresCol="features", labelCol="Survived")

# Entrenar modelo
lr_model = lr.fit(train_df)

# Predecir en conjunto de prueba
predictions = lr_model.transform(test_df)

# Evaluar desempeño del modelo
evaluator = BinaryClassificationEvaluator(labelCol="Survived", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)

print(f"AUC: {auc:.4f}")

AUC: 0.8375


Aprendizaje No Supervisado

In [20]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Definir modelo de clustering con 2 clústeres (porque hay 2 clases en 'Survived')
kmeans = KMeans(featuresCol="features", k=2, seed=42)
kmeans_model = kmeans.fit(df)

# Realizar predicciones (asignación a clúster)
clustered = kmeans_model.transform(df)

# Evaluar cohesión de los clústeres
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(clustered)

print(f"Silhouette Score: {silhouette:.4f}")

Silhouette Score: 0.9744


Interpretaciones Finales 

En caso del modelo de regresion logistica aplicado sobre la muestra se obtubo un AUC de .8375 el AUC deseado es un 1 y algo arriba de .8 indica que el modelo tiene una alta capasidad de distinguir si un pasajero sobrevivio o no. En el caso de k-meeans el indice de siluete de .9744 lo cual respresenta una buena calidad de formacion de clusters Un valor cercano a 1.0 indica que los datos dentro de cada grupo son muy similares entre sí y muy diferentes de los otros grupos. Esto sugiere que las variables seleccionadas para el clustering permitieron diferenciar claramente dos perfiles de pasajeros, sin necesidad de conocer previamente la variable de supervivencia.

